In [ ]:
# READ OH SHEET
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json

# this is setup to work with 4dndcic@gmail.com account
# invite google-sheet-access@fourdn-fourfront.iam.gserviceaccount.com on the google sheet to give access to it

# extract the book id from the google sheet url
#../spreadsheets/d/<book_id>/edit#gid=#######
book_id = '13gFjHcYEwYL2Xpr9-IHtJNYYnYlAQVdyXvCUeSzla6w'
sheet_names = ['DCIC & OH', 'IT', 'NT', 'NOFIC', 'NBC']

# GET KEY FROM S3 
# if you have your own key file, please change the following part accordingly
s3 = boto3.resource('s3')

obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

# The ID and range of a sample spreadsheet.
book = gc.open_by_key(book_id)

for name in sheet_names:
    print()
    print(name)

    worksheet = book.worksheet(name)
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    col_names = df.columns.values

    for index, row in df.iterrows():
        print(row.keys())
        break
    break

    
### More info at https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html

In [ ]:
# Creating new worksheet in the spreadsheet. The name of the worksheet should not already exist otherwise an exception will occur
worksheet = book.add_worksheet(title='New Worksheet', rows='100', cols='100')

In [5]:
# Adding all the data to the spreadsheet at once
gs_write = [] # A list of cell objects to write to the worksheet
# The data that is going to be written in the worksheet. In this example, each key is a column.
my_data = {'Organism':['Human', 'fly', 'chicken', 'mouse'], 'Genome_assembly':['hg38','dm6', 'Galgal5','mm9']}

#Writting the data to the list
row = 1
col = 1
for key in my_data.values():
    row = 1
    for item in key:
        #the gspread.models.Cell method converts cell values into cell objects
        gs_write.append(gspread.models.Cell(row, col, item)) 
        row += 1
    col += 1
#Write the cells to the worksheet
worksheet.update_cells(gs_write)

[<Cell R1C1 'Human'>, <Cell R2C1 'fly'>, <Cell R3C1 'chicken'>, <Cell R4C1 'mouse'>, <Cell R1C2 'hg38'>, <Cell R2C2 'dm6'>, <Cell R3C2 'Galgal5'>, <Cell R4C2 'mm9'>]
